In [1]:
import os

from tqdm import tqdm
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, SoupStrainer

In [2]:
df = pd.read_csv('scielo_dataset.csv', dtype={'ano': np.uint16})  # lendo o ano como um uint16 para economizar memória

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1681 entries, 0 to 1680
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            1672 non-null   object
 1   tipo             1665 non-null   object
 2   ano              1681 non-null   uint16
 3   doi_id           1661 non-null   object
 4   file_bruto_name  1681 non-null   object
dtypes: object(4), uint16(1)
memory usage: 55.9+ KB


Visualizando quais são os tipos dos textos

In [4]:
df['tipo'].value_counts()

artigos gerais                                                                          798
produtos e materiais didáticos                                                          202
pesquisa em ensino de física                                                            158
história da física e ciências afins                                                     145
desenvolvimento em ensino de física                                                      78
notas e discussões                                                                       61
cartas ao editor                                                                         28
seção especial: informática no ensino de física                                          19
editorial                                                                                17
seção especial                                                                           16
seção especial - celebrando os 100 anos de nascimento de richard p. feynman     

Criei essa lista com os tipos que se relacionam com ensino de ciências.

In [5]:
c = ['produtos e materiais didáticos', 'pesquisa em ensino de física', 'história da física e ciências afins',
     'desenvolvimento em ensino de física', 'seção especial: informática no ensino de física', 'história da física',
     'história da física e áreas afins', 'seção especial: encontro " reflexões no ensino de física", sbf/mec ( brasília, 2005)']

Escolhendo os itens que o tipo está contido lista e o ano é maior que 2014

In [6]:
edu_df = df.loc[(df['tipo'].isin(c)) & (df['ano'] > 2014)]  # Analisando de 2015 até 2022

In [7]:
edu_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 323 entries, 50 to 808
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            323 non-null    object
 1   tipo             323 non-null    object
 2   ano              323 non-null    uint16
 3   doi_id           323 non-null    object
 4   file_bruto_name  323 non-null    object
dtypes: object(4), uint16(1)
memory usage: 13.2+ KB


In [8]:
arquivos = edu_df['file_bruto_name'].values  # pegando os arquivos

In [9]:
def limpa_html(text_html) -> str:
     soup = BeautifulSoup(text_html, 'lxml', parse_only=SoupStrainer('div', attrs={'data-anchor': 'Text'}))

     # tags do título, citações, referências, equações e figuras foram removidas.
     remover = soup.find_all('h1') + soup.find_all('span', attrs={'class': 'ref'}) + \
               soup.find_all('div', attrs={'class':'row formula'}) + soup.find_all('div', attrs={'class': 'row fig'}) + \
               soup.find_all('math')

     for tag in remover:
          tag.decompose()

     return ' '.join(soup.text.lower().split())

In [10]:
textos = []
for arq in tqdm(arquivos):
     path = os.path.abspath(os.path.join('artigos_brutos', arq))

     with open(path, 'r', encoding='utf8') as f:
          text = f.read()

     textos.append(limpa_html(text))

100%|██████████| 323/323 [00:23<00:00, 13.82it/s]


# Análise NLP

In [118]:
import spacy
nlp = spacy.load("pt_core_news_sm")

from nltk.corpus import stopwords
from spacy.lang.pt.stop_words import STOP_WORDS

In [119]:
my = {'p', 'fig', 'p.', 'cm', 'et', 'al', 't', 's', 'm', 'v', 'd', 'of', 'the', 'eq'}
stop = set(stopwords.words('portuguese')).union(STOP_WORDS).union(my)

In [120]:
documentos = []
for texto in tqdm(textos):
     doc = nlp(texto)
     documentos.append([token.lemma_ for token in doc if token.orth_ not in stop and token.is_alpha])

100%|██████████| 323/323 [03:59<00:00,  1.35it/s]


In [121]:
from gensim.models import Phrases

# Adiciona bigrams e trigrams para os documentos (só se aparecer 5 vezes ou mais).
bigram = Phrases(documentos, min_count=5)
for idx in range(len(documentos)):
     for token in bigram[documentos[idx]]:
          if '_' in token:  # se for um n-gram, adiciona no documento
               documentos[idx].append(token)

In [127]:
from gensim.corpora import Dictionary

# c
dictionary = Dictionary(documentos)

# Filtra as palabras que ocorrem em menos de 20 documentos ou mais que 60% dos documentos.
dictionary.filter_extremes(no_below=10, no_above=0.6)

In [128]:
corpus = [dictionary.doc2bow(doc) for doc in documentos]

print(f'Número de tokens únicos: {len(dictionary)}')
print(f'Número de documentos: {len(corpus)}')

Número de tokens únicos: 5394
Número de documentos: 323


In [135]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 20
chunksize = 400
passes = 50
iterations = 500
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
     corpus=corpus,
     id2word=id2word,
     chunksize=chunksize,
     alpha='auto',
     eta='auto',
     iterations=iterations,
     num_topics=num_topics,
     passes=passes,
     eval_every=eval_every,
     random_state=99
)

In [136]:
top_topics = model.top_topics(corpus)

avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print(f'Coerência média entre os tópicos: {avg_topic_coherence:.4f}')

Coerência média entre os tópicos: -1.0291


In [137]:
for i, topic in enumerate(top_topics, start=1):
     print(f'Tópico {i} ({topic[1]:.4f}):')
     print('        ', end= '')
     for prob, word, in topic[0]:
          print(f'{word} ({prob:.4f})', end=' ')
     print()

Tópico 1 (-0.6847):
        aprendizagem (0.0129) conteúdo (0.0101) resposta (0.0083) questão (0.0063) prática (0.0055) sala (0.0053) metodologia (0.0048) contexto (0.0046) sala_aula (0.0044) avaliação (0.0044) etapa (0.0042) conceitual (0.0040) estratégia (0.0039) ensino_médio (0.0039) jogo (0.0038) questionário (0.0037) autor (0.0036) compreensão (0.0035) significativo (0.0035) equipe (0.0035) 
Tópico 2 (-0.7870):
        item (0.0368) prova (0.0165) questão (0.0140) alternativa (0.0125) desempenho (0.0097) resposta (0.0082) teste (0.0074) enem (0.0074) média (0.0071) correlação (0.0068) tabela (0.0065) elétrico (0.0065) avaliação (0.0060) ensino_médio (0.0057) instrumento (0.0056) fator (0.0056) acerto (0.0056) concepção (0.0056) variável (0.0055) nota (0.0053) 
Tópico 3 (-0.7998):
        partícula (0.0270) energia (0.0252) quântico (0.0132) interação (0.0117) fóton (0.0095) massa (0.0092) elétron (0.0092) elétrom (0.0086) radiação (0.0073) núcleo (0.0065) átomo (0.0065) carga (0.0